### CellStrat Hub Pack - Deep Learning

#### DL5 - Tensorflow: Keras

In [1]:
from IPython.display import Image
%matplotlib inline

In [2]:
import struct
import sys
import gzip
import shutil
import os
import numpy as np

In [3]:
def load_mnist(path, kind='train'):
    """Load MNIST data from `path`"""
    labels_path = os.path.join(path, 
                               '%s-labels-idx1-ubyte' % kind)
    images_path = os.path.join(path, 
                               '%s-images-idx3-ubyte' % kind)
        
    with open(labels_path, 'rb') as lbpath:
        magic, n = struct.unpack('>II', 
                                 lbpath.read(8))
        labels = np.fromfile(lbpath, 
                             dtype=np.uint8)

    with open(images_path, 'rb') as imgpath:
        magic, num, rows, cols = struct.unpack(">IIII", 
                                               imgpath.read(16))
        images = np.fromfile(imgpath, 
                             dtype=np.uint8).reshape(len(labels), 784)
        images = ((images / 255.) - .5) * 2
 
    return images, labels

In [4]:
X_train, y_train = load_mnist('./', kind='train')
print('Rows: %d,  Columns: %d' %(X_train.shape[0], 
                                 X_train.shape[1]))
X_test, y_test = load_mnist('./', kind='t10k')
print('Rows: %d,  Columns: %d' %(X_test.shape[0], 
                                 X_test.shape[1]))

## mean centering and normalization:
mean_vals = np.mean(X_train, axis=0)
std_val = np.std(X_train)

X_train_centered = (X_train - mean_vals)/std_val
X_test_centered = (X_test - mean_vals)/std_val
 
del X_train, X_test
 
print(X_train_centered.shape, y_train.shape)

print(X_test_centered.shape, y_test.shape)

Rows: 60000,  Columns: 784
Rows: 10000,  Columns: 784
(60000, 784) (60000,)
(10000, 784) (10000,)


In [5]:
import tensorflow as tf
#import tensorflow.contrib.keras as keras
from tensorflow import keras
from tensorflow.keras import layers

np.random.seed(123)
# tf.set_random_seed(123)
tf.compat.v1.set_random_seed(123)

In [6]:
#CellStrat - convert class labels 0-9 into one hot vector using a Keras tool.

In [7]:
y_train_onehot = keras.utils.to_categorical(y_train)
 
print('First 3 labels: ', y_train[:3])
print('\nFirst 3 labels (one-hot):\n', y_train_onehot[:3])

First 3 labels:  [5 0 4]

First 3 labels (one-hot):
 [[0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]]


In [8]:
#CellStrat - now define the 3 layers of the network.

In [9]:
model = keras.models.Sequential()

model.add(
    keras.layers.Dense(
        units=50,    
        input_dim=X_train_centered.shape[1],
        #kernel_initializer='glorot_uniform',
        kernel_initializer='he_normal',
        bias_initializer='zeros',
        activation='tanh'))

model.add(
    keras.layers.Dense(
        units=50,    
        input_dim=50,
        kernel_initializer='glorot_uniform',
        bias_initializer='zeros',
        activation='tanh'))

model.add(
    keras.layers.Dense(
        units=y_train_onehot.shape[1],    
        input_dim=50,
        kernel_initializer='glorot_uniform',
        bias_initializer='zeros',
        activation='softmax'))


sgd_optimizer = keras.optimizers.SGD(
        lr=0.001, decay=1e-7, momentum=.9)

model.compile(optimizer=sgd_optimizer,
              loss='categorical_crossentropy')

In [10]:
#CellStrat - we are using mini-batch stochastic gradient descent here.
#The validation_split parameter is especially handy since it will reserve 10 percent
#of the training data (here, 6,000 samples) for validation after each epoch so that we
#can monitor whether the model is overfitting during training:

In [11]:
history = model.fit(X_train_centered, y_train_onehot,
                    batch_size=64, epochs=50,
                    verbose=1,
                    validation_split=0.1)

Epoch 1/50
844/844 [==============================] - 1s 2ms/step - loss: 0.7225 - val_loss: 0.3717
Epoch 2/50
844/844 [==============================] - 1s 1ms/step - loss: 0.3777 - val_loss: 0.2826
Epoch 3/50
844/844 [==============================] - 1s 1ms/step - loss: 0.3126 - val_loss: 0.2432
Epoch 4/50
844/844 [==============================] - 1s 1ms/step - loss: 0.2754 - val_loss: 0.2185
Epoch 5/50
844/844 [==============================] - 1s 1ms/step - loss: 0.2493 - val_loss: 0.2015
Epoch 6/50
844/844 [==============================] - 1s 1ms/step - loss: 0.2291 - val_loss: 0.1880
Epoch 7/50
844/844 [==============================] - 1s 1ms/step - loss: 0.2129 - val_loss: 0.1773
Epoch 8/50
844/844 [==============================] - 1s 2ms/step - loss: 0.1991 - val_loss: 0.1692
Epoch 9/50
844/844 [==============================] - 1s 2ms/step - loss: 0.1873 - val_loss: 0.1615
Epoch 10/50
844/844 [==============================] - 1s 2ms/step - loss: 0.1769 - val_loss: 0.1551

In [12]:
# CellStrat - let's predict some values using the trained model

In [13]:
y_train_pred = model.predict_classes(X_train_centered, verbose=0)
print('First 3 predictions: ', y_train_pred[:3])

Instructions for updating:
Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
First 3 predictions:  [5 0 4]


In [14]:
#CellStrat - print accuracy of predictions for training data and test data

In [15]:
y_train_pred = model.predict_classes(X_train_centered, 
                                     verbose=0)
correct_preds = np.sum(y_train == y_train_pred, axis=0) 
train_acc = correct_preds / y_train.shape[0]

print('First 3 predictions: ', y_train_pred[:3])
print('Training accuracy: %.2f%%' % (train_acc * 100))

First 3 predictions:  [5 0 4]
Training accuracy: 98.89%


In [16]:
y_test_pred = model.predict_classes(X_test_centered, 
                                    verbose=0)

correct_preds = np.sum(y_test == y_test_pred, axis=0) 
test_acc = correct_preds / y_test.shape[0]
print('Test accuracy: %.2f%%' % (test_acc * 100))

Test accuracy: 96.35%
